In [2]:
"""
Implements Appendix P.1. of Bott et al. (2017), taking in experiment
parameters and a path-integrated magnetic field within the interaction
region to simulate the expected proton flux.
"""
import numpy as np
import scipy as sp
import random
import math
import problem.deflect
from problem.constants import M_PROTON_G, ESU, C_CMS
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [3]:
# 2D histogram colorplot from praline
def hist2D_plot(array, bin_um, title):
    '''
    Genereates the 2D histogram plot based on 2D array
    Parameters
    ----------
    array (2D array): The array that will be plotted on a 2D histogram
    bin_um (float): Length of the side of a bin, in cm
    type (string): type of file input
    title (string): title of the plot
    Returns
    -------
    "title".png (image):2D histogram Plot
    '''
    print ("Constructing " + title + " Plot")
    font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 32,
        }

    num_bins = array.shape[0]
    delta = bin_um/10000.0
    dmax = (delta * num_bins)/2.0

    x = np.zeros((num_bins+1, num_bins+1))
    y = np.zeros((num_bins+1, num_bins+1))
    for i in range(num_bins+1):
        xx = -dmax + i*delta
        for j in range(num_bins+1):
            yy = -dmax + j*delta
            x[i,j] = xx
            y[i,j] = yy

    # Intiating plot
    fig = plt.figure()
    fig.set_figwidth(15.0)
    fig.set_figheight(12.0)
    # Making plot
    ax = fig.add_subplot(1,1,1)
    p = ax.pcolormesh(x,y, array, cmap=cm.afmhot, vmin= array.min(), vmax=array.max())
    ax.set_xlabel("X (cm)", fontdict=font)
    ax.set_ylabel("Y (cm)", fontdict=font)

    xmin = x.min()
    xmax = x.max()
    ymin = y.min()
    ymax = y.max()

    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)

    plt.colorbar(p)

    ax.set_title(title,fontdict=font)
    fig.savefig(title+".png", format='png')

In [4]:
# returns Cartesian velocity components on the unit sphere of radius V
# constrained to a polar angle of theta in [arctan(-lperp/2/ri), 
# arctan(lperp/2/ri)]. This is to constrain proton flux to the 
# interaction region.
def get_vel_init(V, lperp, ri):
    theta = random.uniform(np.arctan(-lperp/2/ri), np.arctan(lperp/2/ri))
    phi = random.uniform(0, 2 * math.pi)
    vx = V * np.sin(theta) * np.cos(phi)
    vy = V * np.sin(theta) * np.sin(phi)
    vz = V * np.cos(theta)
    return [vx, vy, vz]

In [5]:
# returns 2D array of average B values at [x,y] locations.
def get_B(ri, lz, rs, v, X, Y):
    phix = np.loadtxt('solve/phix399000.txt', delimiter=',')
    phiy = np.loadtxt('solve/phiy399000.txt', delimiter=',')
    phin = np.loadtxt('solve/phin399000.txt', delimiter=',')
    wBx, wBy = problem.deflect.reconstruct(ri, lz, rs, v, X, Y, phix, phiy)
    Bx, By = problem.deflect.magpath(wBx, wBy)
    return Bx, By

In [6]:
# loads coordinate grid of the object plane
def get_coord(M):
    # Load the plasma coordinates.
    X = np.loadtxt('solve/plasma_x.txt', delimiter=',')*M
    Y = np.loadtxt('solve/plasma_y.txt', delimiter=',')*M
    return X, Y

In [7]:
# implements 3 by 3 cross product
def cross_product(A, B):
    r = np.zeros(3)
    r[0] = A[1]*B[2] - A[2]*B[1]
    r[1] = -A[0]*B[2] + A[2]*B[0]
    r[2] = A[0]*B[1] - A[1]*B[0]
    return r

In [8]:
# maps this proton to a point in the interaction region, before 
# deflected by B field.
def init_loc(v, lperp, ri):
    vx, vy, vz = v
    t = ri/vz
    x = vx*t
    y = vy*t
    return [x, y, 0]

In [9]:
def to_screen(v, pos, rs):
    t = rs/v[2]
    x = pos[0] + v[0]*t
    y = pos[1] + v[1]*t
    return [x, y, rs]

In [10]:
# v, pos are arrays st v[0] = vx, v[1] = vy, v[2] = vz\
# implements Boris algorithm, as interpreted from 
# https://www.particleincell.com/2011/vxb-rotation/
# where we neglect electric field effects, and consider only deflections
# in the x-y directions
def update_vel_Boris(dt, v, pos, Bx, By):
    B = [Bx, By, 0]
    v_minus = np.zeros(3)
    v_prime = np.zeros(3)
    v_new = np.zeros(3)
    t = np.zeros(3)
    s = np.zeros(3)
    for dim in range (0, 3):
        t[dim] = ESU/M_PROTON_G*B[dim]*.5*dt
    t_mag2 = t[0]**2 + t[1]**2 + t[2]**2
    for dim in range (0, 3):
        s[dim] = 2*t[dim]/(1+t_mag2)
    # because we neglect electric field effects, E = 0
    for dim in range (0, 3):
        v_minus[dim] = v[dim]
    v_minus_cross_t = cross_product(v_minus, t)
    
    for dim in range (0, 3):
        v_prime[dim] = v_minus[dim] + v_minus_cross_t[dim]
    
    v_prime_cross_s = cross_product(v_prime, s)
    
    for dim in range (0, 3):
        v_new[dim] = v_minus[dim] + v_prime_cross_s[dim]
    # advance pos
    
    for dim in range (0, 3):
        pos[dim] = pos[dim] + (v[dim] + v_new[dim])*dt/2
    return pos, v_new
    

In [11]:
# bins screen location to form flux image
def bin_proton():
    pass

In [16]:
lperp = 0.2
dt = .000001



# Plasma parameters, copied from 
ri = 1.3 # Distance from proton source to plasma.
lz = 0.2 # Distance across plasma.
rs = 21.3 # Distance from plasma to screen.
v0 = 5.24e3 # Velocity of protons.
bin_um = 1 # bin size (cm)
M = (rs+ri+.5*lz)/(ri+.5*lz) # magnification factor
X, Y = get_coord(M)



Bx, By = get_B(ri, lz, rs, v0, X, Y)

# x, y positions of protons
pos_x = []
pos_y = []
# load average magnetic field (path-integrated B/lz), create interpolators for B
# in x and y directions
Bx_interp = sp.interpolate.RegularGridInterpolator(
                                    (X[:,0], Y[0,:]),
                                    Bx,
                                    method='linear',
                                    bounds_error=False,
                                    fill_value=0)
By_interp = sp.interpolate.RegularGridInterpolator(
                                    (X[:,0], Y[0,:]),
                                    By,
                                    method='linear',
                                    bounds_error=False,
                                    fill_value=0)
# initialize random set of protons to bin
for p in range (0, 10001):  
    print ("step : "+ str(p))
    v = get_vel_init(v0, lperp, ri)
    print("Iniital v : "+ str(v))
    pos = init_loc(v, lperp, ri)
    toss = False
    while pos[2] < lz :
        print("curr pos: " + str(pos[2]))
        print("curr vel: "+ str(v))
        if abs(pos[0]) >= lperp/2 :
            toss = True
            print("X out of bounds")
            break
        elif abs(pos[1]) >= lperp/2 :
            toss = True
            print("Y out of bounds")
            break
        elif  v[2] < 0 :
            toss = True
            print("Z velocity negative")
            break
        Bx = Bx_interp((pos[0], pos[1]))
        By = By_interp((pos[0], pos[1]))
        new_pos, new_v = update_vel_Boris(dt, v, pos, Bx, By)
        # print("NEW POSITION "+str(new_pos))
        pos = new_pos
        v = new_v
        # print("UPDATING POSITION "+str(pos))
        
    if toss:
        continue
    
    screen_pos = to_screen(v, pos, rs)
    if abs(screen_pos[0]) > M*lperp/2 or abs(screen_pos[1]) > M*lperp/2:
        continue
    print("ACCESSED")
    pos_x.append(screen_pos[0])
    pos_y.append(screen_pos[1])
    
# binning

print('Histogramming protons...')

flux_image = np.histogram2d(pos_x, pos_y, bins=X.shape)
    
print('DONE')

step : 0
Iniital v : [-60.34838064965799, -268.524922981366, 5232.767187510908]
curr pos: 0
curr vel: [-60.34838064965799, -268.524922981366, 5232.767187510908]
curr pos: -8.28714037197642e-11
curr vel: [   58.8580426    268.85229689 -5232.76735325]
Z velocity negative
step : 1
Iniital v : [-135.87431965415678, -172.2249869667183, 5235.406070509033]
curr pos: 0
curr vel: [-135.87431965415678, -172.2249869667183, 5235.406070509033]
curr pos: -8.278177392639918e-11
curr vel: [  135.41849675   172.57860247 -5235.40623607]
Z velocity negative
step : 2
Iniital v : [66.39152834203216, -128.37536758506974, 5238.00648433751]
curr pos: 0
curr vel: [66.39152834203216, -128.37536758506974, 5238.00648433751]
curr pos: -8.290484493045368e-11
curr vel: [  -66.27987844   128.42628519 -5238.00665015]
Z velocity negative
step : 3
Iniital v : [204.46753982676782, -184.68605705364544, 5232.7511010449525]
curr pos: 0
curr vel: [204.46753982676782, -184.68605705364544, 5232.7511010449525]
curr pos: -8.2995

curr pos: -8.31128654681379e-11
curr vel: [  -72.19373311   258.96995165 -5233.09875973]
Z velocity negative
step : 206
Iniital v : [-53.99756736229228, 0.5243558525496931, 5239.721747170349]
curr pos: 0
curr vel: [-53.99756736229228, 0.5243558525496931, 5239.721747170349]
curr pos: -8.371786771022016e-11
curr vel: [ 5.39449549e+01 -2.04925206e+00 -5.23972191e+03]
Z velocity negative
step : 207
Iniital v : [87.50361900287497, -55.1624626445019, 5238.97892908307]
curr pos: 0
curr vel: [87.50361900287497, -55.1624626445019, 5238.97892908307]
curr pos: -8.312089812534395e-11
curr vel: [  -86.8404584     56.18517183 -5238.97909532]
Z velocity negative
step : 208
Iniital v : [14.662946295414294, 77.97431778525637, 5239.399297989404]
curr pos: 0
curr vel: [14.662946295414294, 77.97431778525637, 5239.399297989404]
curr pos: -8.288178469229024e-11
curr vel: [  -15.4178775    -77.81740344 -5239.39946375]
Z velocity negative
step : 209
Iniital v : [-164.8189059530239, -22.051064175421313, 5237.3

curr vel: [-5.883014385956652, -7.070477217086921, 5239.991927330962]
curr pos: -8.933957815315807e-11
curr vel: [ 4.39051706e+00  7.96569592e+00 -5.23999211e+03]
Z velocity negative
step : 358
Iniital v : [41.46914606530758, -163.41738368484735, 5237.286995060917]
curr pos: 0
curr vel: [41.46914606530758, -163.41738368484735, 5237.286995060917]
curr pos: -8.343757417605957e-11
curr vel: [  -46.27509644   162.1166212  -5237.28716194]
Z velocity negative
step : 359
Iniital v : [-19.050672347936015, -8.059583426138035, 5239.959171119379]
curr pos: 0
curr vel: [-19.050672347936015, -8.059583426138035, 5239.959171119379]
curr pos: -8.645119123684708e-11
curr vel: [   19.03400685     7.98621202 -5239.95934402]
Z velocity negative
step : 360
Iniital v : [314.94730746590466, -80.30909012808819, 5229.9100033903815]
curr pos: 0
curr vel: [314.94730746590466, -80.30909012808819, 5229.9100033903815]
curr pos: -8.279520079668145e-11
curr vel: [ -314.91836676    80.41173218 -5229.91016898]
Z veloci

Iniital v : [-253.07703697083582, -13.887408202048992, 5233.866558601919]
curr pos: 0
curr vel: [-253.07703697083582, -13.887408202048992, 5233.866558601919]
curr pos: -8.278704626718535e-11
curr vel: [  253.06084597    14.11813723 -5233.86672418]
Z velocity negative
step : 517
Iniital v : [-17.69929159661046, -95.52980660621814, 5239.09923470884]
curr pos: 0
curr vel: [-17.69929159661046, -95.52980660621814, 5239.09923470884]
curr pos: -8.282860562758288e-11
curr vel: [   14.50339139    96.05792379 -5239.09940037]
Z velocity negative
step : 518
Iniital v : [-111.34616752705438, -241.85090323676914, 5233.231331747202]
curr pos: 0
curr vel: [-111.34616752705438, -241.85090323676914, 5233.231331747202]
curr pos: -8.271472597698448e-11
curr vel: [  110.25329039   242.3474962  -5233.23149718]
Z velocity negative
step : 519
Iniital v : [-58.0652571436654, 109.18352038076075, 5238.5405777554215]
curr pos: 0
curr vel: [-58.0652571436654, 109.18352038076075, 5238.5405777554215]
curr pos: -8.38

curr vel: [  103.19898416  -232.24643453 -5233.83335265]
Z velocity negative
step : 690
Iniital v : [94.30394276415748, -173.60550704859608, 5236.274237881505]
curr pos: 0
curr vel: [94.30394276415748, -173.60550704859608, 5236.274237881505]
curr pos: -8.274277843156597e-11
curr vel: [  -93.36246354   174.10865299 -5236.27440337]
Z velocity negative
step : 691
Iniital v : [-45.98730288696245, -47.72246114755125, 5239.580873951199]
curr pos: 0
curr vel: [-45.98730288696245, -47.72246114755125, 5239.580873951199]
curr pos: -8.276682183350203e-11
curr vel: [   44.74024307    48.87577437 -5239.58103948]
Z velocity negative
step : 692
Iniital v : [11.704278096777259, -42.32413427118512, 5239.815996533888]
curr pos: 0
curr vel: [11.704278096777259, -42.32413427118512, 5239.815996533888]
curr pos: -8.028519732761197e-11
curr vel: [  -13.68541198    41.7055062  -5239.8161571 ]
Z velocity negative
step : 693
Iniital v : [234.52410168880118, -138.08395465878334, 5232.927600033546]
curr pos: 0
cu

...

curr pos: -8.280950441985623e-11
curr vel: [  237.71658362   257.31787586 -5228.27680375]
Z velocity negative
step : 9230
Iniital v : [-207.06590620508962, 178.65563246307903, 5232.858289259961]
curr pos: 0
curr vel: [-207.06590620508962, 178.65563246307903, 5232.858289259961]
curr pos: -8.281834925583098e-11
curr vel: [  207.28153126  -178.40055442 -5232.8584549 ]
Z velocity negative
step : 9231
Iniital v : [252.15419389292683, 117.60114812538033, 5232.608167296859]
curr pos: 0
curr vel: [252.15419389292683, 117.60114812538033, 5232.608167296859]
curr pos: -8.32809755593189e-11
curr vel: [ -251.86945882  -118.20236946 -5232.60833386]
Z velocity negative
step : 9232
Iniital v : [314.8366283208714, -162.810377910735, 5227.9987259287]
curr pos: 0
curr vel: [314.8366283208714, -162.810377910735, 5227.9987259287]
curr pos: -8.249936217907816e-11
curr vel: [ -315.43477989   161.64311337 -5227.99889093]
Z velocity negative
step : 9233
Iniital v : [198.48987524037878, 12.56260409706701, 5236.

step : 9433
Iniital v : [173.41142900768196, 135.12761672945607, 5235.386232503437]
curr pos: 0
curr vel: [173.41142900768196, 135.12761672945607, 5235.386232503437]
curr pos: -8.308174892590613e-11
curr vel: [ -173.74958895  -134.6860683  -5235.38639867]
Z velocity negative
step : 9434
Iniital v : [-17.94470127791903, -157.06601773287144, 5237.614748506192]
curr pos: 0
curr vel: [-17.94470127791903, -157.06601773287144, 5237.614748506192]
curr pos: -8.250791142927482e-11
curr vel: [   15.48094026   157.32246547 -5237.61491352]
Z velocity negative
step : 9435
Iniital v : [-8.05000160793964, 12.542311984621518, 5239.978806052959]
curr pos: 0
curr vel: [-8.05000160793964, 12.542311984621518, 5239.978806052959]
curr pos: -8.050414362514857e-11
curr vel: [ 5.23351778e+00 -1.38937053e+01 -5.23997897e+03]
Z velocity negative
step : 9436
Iniital v : [-6.530152798583662, 102.21188073090003, 5238.998958631494]
curr pos: 0
curr vel: [-6.530152798583662, 102.21188073090003, 5238.998958631494]
cur

curr pos: -8.324663212988525e-11
curr vel: [   97.22508725    65.08448061 -5238.69366281]
Z velocity negative
step : 9606
Iniital v : [135.53459493876238, 107.86818438489729, 5237.136128493538]
curr pos: 0
curr vel: [135.53459493876238, 107.86818438489729, 5237.136128493538]
curr pos: -8.339044597960309e-11
curr vel: [ -136.54643141  -106.57624871 -5237.13629527]
Z velocity negative
step : 9607
Iniital v : [46.48348728522852, 285.28342667980326, 5232.021851241796]
curr pos: 0
curr vel: [46.48348728522852, 285.28342667980326, 5232.021851241796]
curr pos: -8.253675878222566e-11
curr vel: [  -47.23942554  -285.15619837 -5232.02201632]
Z velocity negative
step : 9608
Iniital v : [100.05160756843044, 255.89441699382093, 5232.791580330175]
curr pos: 0
curr vel: [100.05160756843044, 255.89441699382093, 5232.791580330175]
curr pos: -8.325168710143771e-11
curr vel: [  -99.11172098  -256.25651413 -5232.79174683]
Z velocity negative
step : 9609
Iniital v : [94.29304900902957, -9.955072367046773, 

curr vel: [  -17.62803957    12.40150583 -5239.95567299]
Z velocity negative
step : 9769
Iniital v : [84.5487372064231, -271.6613628465685, 5232.270216165555]
curr pos: 0
curr vel: [84.5487372064231, -271.6613628465685, 5232.270216165555]
curr pos: -8.303051527036586e-11
curr vel: [  -83.88394205   271.86417848 -5232.27038223]
Z velocity negative
step : 9770
Iniital v : [126.86266840490016, -226.56295986143823, 5233.562370755126]
curr pos: 0
curr vel: [126.86266840490016, -226.56295986143823, 5233.562370755126]
curr pos: -8.319405151269165e-11
curr vel: [ -126.22850045   226.9130571  -5233.56253714]
Z velocity negative
step : 9771
Iniital v : [111.92805294213213, 144.1128022457329, 5236.8218998924785]
curr pos: 0
curr vel: [111.92805294213213, 144.1128022457329, 5236.8218998924785]
curr pos: -8.3306677879591e-11
curr vel: [ -111.70684469  -144.27828876 -5236.82206651]
Z velocity negative
step : 9772
Iniital v : [59.056369646492314, 98.84168932687551, 5238.734834447644]
curr pos: 0
curr

curr vel: [  195.35139175   343.32788316 -5225.08983639]
Z velocity negative
step : 9961
Iniital v : [6.672341672962991, 13.143751187338411, 5239.97926729308]
curr pos: 0
curr vel: [6.672341672962991, 13.143751187338411, 5239.97926729308]
curr pos: -7.62249501349288e-11
curr vel: [   -8.42360488   -12.03011018 -5239.97941974]
Z velocity negative
step : 9962
Iniital v : [-71.01493418820911, 175.81574818603372, 5236.568122521851]
curr pos: 0
curr vel: [-71.01493418820911, 175.81574818603372, 5236.568122521851]
curr pos: -8.313331818499136e-11
curr vel: [   70.80054883  -175.89724029 -5236.56828879]
Z velocity negative
step : 9963
Iniital v : [-99.87650888192043, 8.396358427438642, 5239.0413420910045]
curr pos: 0
curr vel: [-99.87650888192043, 8.396358427438642, 5239.0413420910045]
curr pos: -8.337593862961512e-11
curr vel: [   99.88344798    -8.20764528 -5239.04150884]
Z velocity negative
step : 9964
Iniital v : [-77.0829865504849, -98.75042262679828, 5238.502321008886]
curr pos: 0
curr 

In [34]:
pos_x

[]

In [ ]:
hist2D_plot(flux_image, bin_um, "boris flux")